## Definição:
Os carregadores de documentos são componentes do LangChain que facilitam o carregamentos de arquivos/documentos para que eles possam interagir  com outros componentes de LangChain.
Eles não são utilizados diretamente dentro das cadeias (chains), mas são peças fundamentais no pré processamento dos dados para que você possa adicionar informações à sua chain (mais especifico nos prompts) ou indexar informações em bancos vetoriais.

Os carregadores de documentos são projetados para carregar objetos de documentos. O LangChain tem centenas de integrações com várias fontes de dados para carregar dados de: Slack, Notion, Google Drive, etc. ([link](https://python.langchain.com/docs/integrations/document_loaders/))

Cada Document Loader tem seus próprios parâmetros específicos, mas todos eles podem ser invocados da mesma maneira com o método `.load`(método padrão de carregamento) ou `.lazy_load` (usado para trabalhar com grandes conjuntos de dados).
[Documentação](https://python.langchain.com/docs/how_to/#document-loaders)

### Tipos

##### Carregadores de PDF:

[link](https://python.langchain.com/docs/how_to/document_loader_pdf/)
#### Simples Carregamento de texto:

Para fazer a leitura de texto presente em PDFs, podemos usar a biblioteca pypdf.
 


- Versão assíncrona:

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
# Importa conectando com o cliente OpenAI
from helpers.llm import initialize_llm, logger, pretty_print_docs

# Parte 1 - Importando os componentes do LangChain
llm, _, _ = initialize_llm()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader  
import asyncio  # Import necessário para gerenciar o loop de eventos  
  
  
# Define uma função assíncrona para realizar a tarefa  
async def process_pdf():  
    loader = PyPDFLoader(r"data/DENGUE.pdf")  
    pages_asinc = []  
  
    # Para leitura assíncrona  
    async for page in loader.alazy_load():  
        pages_asinc.append(page)  
  
    # Imprime o resultado  
    pretty_print_docs(pages_asinc)  
  
  
# Executa a função assíncrona  
await process_pdf()
# if __name__ == "__main__":  
#     await asyncio.gather(process_pdf())

- Versão síncrona:


In [ ]:
from langchain_community.document_loaders import PyPDFLoader  
  
loader = PyPDFLoader(r"data/DENGUE.pdf")  
pages_sinc = loader.load()  
  
# Aqui verificaremos que pages_sinc é uma lista de 'Document':  
print("\n------ Imprimindo o resultado ------\n")  
print(pages_sinc)  
  
print("\n------ Imprimindo o resultado ------\n")  

pages = []
# Acessando os valores carregados:  
for elemento in pages_sinc:  
    print("----- Página Inicio -----")  
    print(f"Conteúdo: {elemento.page_content}\n")  
    print(f"Metadado: {elemento.metadata}")  
    print("----- Página Fim    -----")


# async for page in loader.alazy_load():
#     pages.append(page)     

# print(f"-> {pages[0].metadata}\n")
# print(pages[0].page_content)     

Para PDFs não estruturados, com imagens ou tabelas, é necessário extratores mais complexos e robustos, então convido você a acessar a documentação do LangChain onde há vários exemplos.

## Carregador de Texto

In [ ]:
from langchain_community.document_loaders import TextLoader  
  
loader = TextLoader(r"data/exemplo_arquivo.txt")  
pages_sinc = loader.load()  
  
# Aqui verificaremos que pages_sinc é uma lista de 'Document':  
print("\n------ Imprimindo o resultado ------\n")  
pretty_print_docs(pages_sinc)  



------ Imprimindo o resultado ------

Document 1:

A 2015 paper, “Neural Machine Translation by Jointly Learning to Align and Translate”,
introduced a new approach to addressing this bottleneck. Rather than having
the encoder supply a single thought vector, it preserved all the hidden state vectors
generated for each token encountered in the encoding process and then allowed
the decoder to “soft search” over all of the vectors. As a demonstration, the paper
showed that using soft search with an English-to-French translation model increased
translation quality significantly. This soft search technique soon came to be known as
the attention mechanism.
The attention mechanism soon gained a good deal of attention of its own in the
AI community, culminating in the 2017 Google Research paper “Attention Is All
You Need”, which introduced the transformer architecture shown in Figure 1-4. The
transformer retained the high-level structure of its predecessor—consisting of an
encoder that receive

## Carregador de Página Web
[link](https://python.langchain.com/docs/how_to/document_loader_web/)

Para fazer a leitura de páginas da web no formato LangChain, podemos utilizar a biblioteca `beautifulsoup4` ou `langchain-unstructured`.

Importações:

In [ ]:
%pip install langchain-community langchain-unstructured unstructured beautifulsoup4 

### Para uma análise simples:


In [ ]:
from langchain_community.document_loaders import WebBaseLoader  
  
page_url = "https://python.langchain.com/docs/introduction/"  
  
loader = WebBaseLoader(web_paths=[page_url])  
resultado = loader.load()  
  
  
# Aqui verificaremos que pages_sinc é uma lista de 'Document':  
print("\n------ Imprimindo o resultado ------\n")  
print(resultado)  
  
print("\n------ Imprimindo o resultado ------\n")  
# Acessando os valores carregados:  
for elemento in resultado:  
    print("----- Página Inicio -----")  
    print(f"Conteúdo: {elemento.page_content}\n")  
    print(f"Metadado: {elemento.metadata}")  
    print("----- Página Fim    -----")

### Para uma análise avançada:

Este método é apropriado se quisermos um controle ou processamento mais granular do conteúdo da página. Abaixo, em vez de gerar um `Document`por página e controlar seu conteúdo via BeautifulSoup, geramos vários `Document`objetos representando estruturas distintas em uma página. Essas estruturas podem incluir títulos de seção e seus textos de corpo correspondentes, listas ou enumerações, tabelas e muito mais.

In [ ]:
from langchain_unstructured import UnstructuredLoader  
  
page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"  
loader = UnstructuredLoader(web_url=page_url)  
  
resultado = loader.load()  
  
  
# Aqui verificaremos que pages_sinc é uma lista de 'Document':  
print("\n------ Imprimindo o resultado ------\n")  
print(resultado)  
  
print("\n------ Imprimindo o resultado ------\n")  
# Acessando os valores carregados:  
for elemento in resultado:  
    print("----- Página Inicio -----")  
    print(f"Conteúdo: {elemento.page_content}\n")  
    print(f"Metadado: {elemento.metadata}")  
    print("----- Página Fim    -----")

## Carregadores de CSVs
[link](https://python.langchain.com/docs/how_to/document_loader_csv/)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader  
  
file_path = r"exemplo_arquivo.csv"  
  
loader = CSVLoader(file_path=file_path)  
data = loader.load()  
  
linha = 0  
for record in data:  
    print(f"Imprimindo linha: {linha}")  
    print(f"-----------")  
    print(record)  
    print(f"-----------")  
    linha+=1  
print("---- Imprimindo de forma estruturada ----")  
for record in data:  
    print(f"Imprimindo linha: {linha}")  
    print(f"Conteúdo: {record.page_content}\n")  
    print(f"-----------")  
    linha+=1

## Carregadores de JSON
[link](https://python.langchain.com/docs/how_to/document_loader_json/)

In [ ]:
%pip install jq

In [ ]:
from langchain_community.document_loaders import JSONLoader


loader = JSONLoader(
    file_path='exemplo_arquivo.json',
    jq_schema='.messages[].content',
    text_content=False)

data = loader.load()

for record in data:  
    print(f"Conteúdo: {record.page_content}\n")  
    print(f"-----------")  
    

## Carregadores de Markdown
[link](https://python.langchain.com/docs/how_to/document_loader_markdown/)

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

markdown_path = "Ambiente.md"
loader = UnstructuredMarkdownLoader(markdown_path)

data = loader.load()
assert len(data) == 1
assert isinstance(data[0], Document)
readme_content = data[0].page_content
print(readme_content[:250])
